# **텍스트 분류 모델 파인 튜닝하기**

# **1. 환경준비**

## (1) 라이브러리 설치


In [ ]:
!pip install transformers==4.31.0

In [ ]:
!pip install datasets

* 설치후 세션 재시작

## (2) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datasets import load_dataset  # 데이터셋 다운로드


## (3) 데이터셋 다운로드

* emotions 데이터셋 소개
    * 트위터 글(텍스트) 기반의 감정 분류를 위한 데이터셋
    * 데이터셋 구조
        * text: 감정을 분석할 텍스트 데이터.
        * label: 'sadness', 'joy', 'love', 'anger', 'fear', 'surprise'
        * input_ids와 attention_mask: transformers 라이브러리와 함께 모델을 학습시킬 때 사용되는, 텍스트를 모델이 처리할 수 있는 형태로 인코딩한 값들입니다.
        * train 16,000건, val 2,000건, test 2,000건

In [ ]:
# emotion 데이터셋 다운로드
imdb = load_dataset("imdb")

# # 데이터 줄이기
imdb["train"] = imdb["train"].shuffle(seed=42).select(range(10000))
imdb["test"] = imdb["test"].shuffle(seed=42).select(range(3000))
imdb["unsupervised"] = imdb["unsupervised"].shuffle(seed=42).select(range(1))  # 사용하지 않음

In [ ]:
# 데이터 구조
imdb

In [ ]:
# 데이터 레이블
classes = imdb['train'].features['label'].names
classes

# **2.데이터 둘러보기**

## (1) 데이터 프레임으로 변환

In [ ]:
# 데이터프레임으로 변환
imdb.set_format(type="pandas")

# train 데이터 만 추출
df = imdb["train"][:]

# 정수인코딩된 레이블에 원래 문자 추가하기
def label_int2str(row):
    return imdb["train"].features["label"].int2str(row)

df["label_name"] = df["label"].apply(label_int2str)
df.head()

## (2) 클래스 분포 살펴보기

In [ ]:
df['label_name'].value_counts()

In [ ]:
sns.countplot(x = 'label_name', data = df)
plt.grid()
plt.show()

## (3) 트윗 문장 길이(단어 수) 분포 확인

* 트랜스포머 모델은 최대 문맥 길이라는 최대 입력 시퀀스 길이가 있음
* DistilBERT 는 최대 문맥 크기가 512 토큰.
* 토큰을 단어단위로 간주할 때, 트윗당 단어 분포

In [ ]:
df["Words Per Sentence"] = df["text"].str.split().apply(len)
sns.histplot(x = 'Words Per Sentence', data = df, bins = 30)
plt.grid()
plt.show()

In [ ]:
sns.kdeplot(x = 'Words Per Sentence', data = df, hue = 'label_name', common_norm = False)
plt.grid()
plt.show()

* 이제 더이상 데이터프레임 포멧이 필요하지 않으니 원본으로 되돌려 놓자.

In [ ]:
imdb.reset_format()

# **3.데이터 준비**

* BERT에서 사용되는 토크나이즈 WordPiece
* AutoTokenizer 클래스 : 체크포인트 이름을 사용해 모델의 설정, 사전훈련된 가중치, 어휘사전을 자동으로 추출


In [ ]:
from transformers import AutoTokenizer

## (1) 토크나이저 다운로드
* bert 모델 학습시 생성된 토크나이저 다운로드

* 모델 입력을 위한 필드 이름

## **(2) 데이터셋 토큰화**

In [ ]:
# 문장 하나씩 토크나이즈 하기 위한 함수 생성
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

* map 매서드는 말뭉치에 있는 모든 샘플을 개별적으로 적용.

In [ ]:
imdb_encoded =

In [ ]:
# 데이터 한건에 대한 내용을 살펴봅시다.
col_names = imdb_encoded["train"].column_names
sample_data = imdb_encoded["train"][0]
for i in col_names :
    print(i + ' :', sample_data[i])

## (3) 텐서플로 학습을 위한 데이터 구성

In [ ]:
# 배치에 포함될 샘플의 수
batch_size =

# 필요한 칼럼 : ['input_ids', 'attention_mask']
token_cols =

# 데이터셋 구성
train =

test =


In [ ]:
train

# **4.파인튜닝**

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
from keras.optimizers import Adam
from sklearn.metrics import *

## **(1) 사전 훈련된 모델 로드하기**

* 이진 분류 : 클래스 수 2
* loss : binary_crossentropy

In [ ]:
# 사전훈련된 모델 지정
preTrModel = "distilbert-base-uncased"

# Output Layer 노드 수
nclass =

# 모델 로드하기
model_ft =

## **(2) 추가 학습**


In [ ]:
# 컴파일 및 학습


## **(3) 예측 및 평가**

# **5.모델 저장**

* 구글 드라이브에 모델 저장하기
    * 구글 드라이브 연결
    * 구글 드라이브에 fine_tuned 폴더 생성
    * 저장할 경로 지정 : /content/drive/MyDrive/fine_tuned/bert_imdb_fine_tuned

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# model 저장하기



* 모델 저장 후, 허깅페이스에 업로드
    * 허깅페이스 회원가입
    * 새 모델을 눌러 저장소를 생성하고,
    * 구글 드라이브의 모델 파일들을 업로드

# **6.모델 사용**

* 허깅페이스에 등록된 모델은 pipeline을 통해 사용할 수 있습니다.